# 1. Import needed libraries

In [1]:
import os
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
#---------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
#---------------------------------------
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#---------------------------------------
import warnings
warnings.filterwarnings("ignore")

2024-03-11 08:32:26.587301: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-11 08:32:26.587425: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-11 08:32:26.726494: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# 2. Preprocessing

## 2.1 Load data

In [2]:
'''class_dict = tr_gen.class_indices
classes = list(class_dict.keys())
images, labels = next(ts_gen)

plt.figure(figsize=(20, 20))

for i, (image, label) in enumerate(zip(images, labels)):
    plt.subplot(4,4, i + 1)
    plt.imshow(image)
    class_name = classes[np.argmax(label)]
    plt.title(class_name, color='k', fontsize=15)

plt.show()'''

"class_dict = tr_gen.class_indices\nclasses = list(class_dict.keys())\nimages, labels = next(ts_gen)\n\nplt.figure(figsize=(20, 20))\n\nfor i, (image, label) in enumerate(zip(images, labels)):\n    plt.subplot(4,4, i + 1)\n    plt.imshow(image)\n    class_name = classes[np.argmax(label)]\n    plt.title(class_name, color='k', fontsize=15)\n\nplt.show()"

# 3. Building Deep Learning Model

In [3]:
img_rows, img_cols = 128, 128
input_shape = (img_rows, img_cols, 3)

#n_classes = df['category'].nunique()
n_classes = 4
print('Total number of unique categories:', n_classes)

from os import listdir, makedirs
from os.path import isfile, join, basename, splitext, isfile, exists

import numpy as np
import pandas as pd

from tqdm import tqdm_notebook

import tensorflow as tf
import keras.backend as K

import keras
from keras.models import Sequential, Model
from keras.layers import Dropout, Dense, Flatten, BatchNormalization
from keras.layers import DepthwiseConv2D, SeparableConvolution2D, Convolution2D, Conv2D,GRU, LSTM, AlphaDropout, Embedding, ZeroPadding2D,AveragePooling2D, MaxPooling2D, GlobalAveragePooling2D, GlobalMaxPooling2D, Dropout
from keras.layers import Concatenate, Average, Maximum, Bidirectional, TimeDistributed
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
#from keras.engine.input_layer import Input
from keras.models import load_model
#from keras.initializers import LecunNormal

import matplotlib.pyplot as plt
import seaborn as sns

#pd.set_option('precision', 30)
np.set_printoptions(precision = 30)


#tf.set_random_seed(1090)

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as img

import cv2
import itertools
import pathlib
import warnings
from PIL import Image
from random import randint
warnings.filterwarnings('ignore')

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef as MCC
from sklearn.metrics import balanced_accuracy_score as BAS
from sklearn.metrics import classification_report, confusion_matrix


from tensorflow import keras
from keras import layers
import tensorflow as tf
#import tensorflow_addons as tfa
from tensorflow.keras.preprocessing import image_dataset_from_directory
##from keras.utils.vis_utils import plot_model
from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Conv2D, Flatten
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator as IDG
from tensorflow.keras.layers import SeparableConv2D, BatchNormalization, GlobalAveragePooling2D

from distutils.dir_util import copy_tree, remove_tree

import os
#print(os.listdir("../input/alzheimer-mri-dataset/Dataset"))
import tensorflow as tf
from keras.datasets import mnist
import cv2
import os
import pathlib
from keras.layers import Conv2D, Conv2DTranspose,Concatenate, Dropout, Dense, Reshape, LayerNormalization, LeakyReLU
from keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import f1_score, recall_score, precision_score
print("TensorFlow Version:", tf.__version__)

Total number of unique categories: 4
TensorFlow Version: 2.15.0


dataset 1: https://www.kaggle.com/datasets/masoudnickparvar/brain-tumor-mri-dataset

In [4]:
class ReadDataset:
    def __init__(self, datasetpath, labels, image_shape):
        self.datasetpath = datasetpath
        self.labels = labels
        self.image_shape = image_shape
    def returListImages(self,):
        self.images = []
        for label in self.labels:
            self.images.append(list(pathlib.Path(os.path.join(self.datasetpath,
                                                              label)).glob('*.*')))
    def readImages(self,):
        self.returListImages()
        self.finalImages = []
        labels = []
        for label in range(len(self.labels)):
            for img in self.images[label]:
                img = cv2.imread(str(img))
                img = cv2.resize(img , self.image_shape)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img  = img/255
                self.finalImages.append(img)
                labels.append(label)
        images = np.array(self.finalImages)
        labels = np.array(labels)
        return images, labels
#/kaggle/input/alzheimer-mri-dataset/Dataset
readDatasetObject = ReadDataset('/kaggle/input/brain-tumor-mri-dataset/Training',
                               ['glioma', 'meningioma','notumor', 'pituitary'],
                               (128,128))
images_train, labels_train = readDatasetObject.readImages()
images_train.shape, labels_train.shape

((5712, 128, 128, 3), (5712,))

In [9]:
'''
X_val.shape, X_test.shape, y_val.shape, y_test.shape
'''

from sklearn.preprocessing import OneHotEncoder
labels_train_mri = labels_train.reshape(-1, 1)
encoder = OneHotEncoder(sparse=False)

# Fit and transform the data
labels_train_mri = encoder.fit_transform(labels_train_mri)
labels_train_mri.shape


(5712, 4)

In [5]:
readDatasetObject = ReadDataset('/kaggle/input/brain-tumor-mri-dataset/Testing',
                               ['glioma', 'meningioma','notumor', 'pituitary'],
                               (128,128))
images_test, labels_test = readDatasetObject.readImages()
images_test.shape, labels_test.shape

((1311, 128, 128, 3), (1311,))

In [6]:
from sklearn.model_selection import train_test_split

X_val_mri, X_test_mri, y_val_mri, y_test_mri = train_test_split(images_test, labels_test, 
                                                    test_size=0.5, random_state=42)
X_val_mri.shape, X_test_mri.shape, y_val_mri.shape, y_test_mri.shape


((655, 128, 128, 3), (656, 128, 128, 3), (655,), (656,))

In [8]:
'''
X_val.shape, X_test.shape, y_val.shape, y_test.shape
'''

from sklearn.preprocessing import OneHotEncoder
y_test_mri = y_test_mri.reshape(-1, 1)
encoder = OneHotEncoder(sparse=False)

# Fit and transform the data
y_test_mri = encoder.fit_transform(y_test_mri)
y_test_mri.shape

(656, 4)

In [7]:
'''
X_val.shape, X_test.shape, y_val.shape, y_test.shape
'''

from sklearn.preprocessing import OneHotEncoder
y_val_mri = y_val_mri.reshape(-1, 1)
encoder = OneHotEncoder(sparse=False)

# Fit and transform the data
y_val_mri = encoder.fit_transform(y_val_mri)
y_val_mri.shape

(655, 4)

In [8]:
images_ct.shape, labels_ct.shape

((2501, 128, 128, 3), (2501,))

dataset 2: https://www.kaggle.com/datasets/afridirahman/brain-stroke-ct-image-dataset

In [10]:
readDatasetObject = ReadDataset('/kaggle/input/brain-stroke-ct-image-dataset/Brain_Data_Organised',
                               ['Normal', 'Stroke'],
                               (128,128))
images_ct, labels_ct = readDatasetObject.readImages()
images_ct.shape, labels_ct.shape

((2501, 128, 128, 3), (2501,))

In [15]:
from sklearn.model_selection import train_test_split

X_train_ct, X_test_ct, y_train_ct, y_test_ct = train_test_split(images_ct, labels_ct, test_size=0.2, random_state=42)
X_train_ct.shape, X_test_ct.shape, y_train_ct.shape, y_test_ct.shape

((2000, 128, 128, 3), (501, 128, 128, 3), (2000,), (501,))

In [27]:
X_test_ct.shape, y_test_ct.shape

((656, 128, 128, 3), (656,))

In [16]:
y_test_ct = np.concatenate((y_test_ct, y_test_ct, y_test_ct), axis=0)
X_test_ct = np.concatenate((X_test_ct, X_test_ct, X_test_ct), axis=0)
X_test_ct.shape, y_test_ct.shape

((1503, 128, 128, 3), (1503,))

In [17]:
random_indices = np.random.choice(1503, 1311, replace=False)
X_test_ct = X_test_ct[random_indices]
y_test_ct = y_test_ct[random_indices]
X_test_ct.shape, y_test_ct.shape

((1311, 128, 128, 3), (1311,))

In [18]:
from sklearn.model_selection import train_test_split

X_val_ct, X_test_ct, y_val_ct, y_test_ct = train_test_split(X_test_ct, y_test_ct, 
                                                test_size=0.5, random_state=42)
X_val_ct.shape, X_test_ct.shape, y_val_ct.shape, y_test_ct.shape


((655, 128, 128, 3), (656, 128, 128, 3), (655,), (656,))

In [29]:
'''
X_val.shape, X_test.shape, y_val.shape, y_test.shape
'''

from sklearn.preprocessing import OneHotEncoder
y_val_ct = y_val_ct.reshape(-1, 1)
encoder = OneHotEncoder(sparse=False)

# Fit and transform the data
y_val_ct = encoder.fit_transform(y_val_ct)
y_val_ct.shape

(655, 2)

In [30]:
'''
X_val.shape, X_test.shape, y_val.shape, y_test.shape
'''

from sklearn.preprocessing import OneHotEncoder
y_test_ct = y_test_ct.reshape(-1, 1)
encoder = OneHotEncoder(sparse=False)

# Fit and transform the data
y_test_ct = encoder.fit_transform(y_test_ct)
y_test_ct.shape

(656, 2)

## data augmentaion for dataset2 to match samples size with dataset 1

In [19]:
import numpy as np
import cv2

def rotate_image(image, angle):
    """
    Rotate the image by the specified angle.
    """
    center = tuple(np.array(image.shape[1::-1]) / 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated_image = cv2.warpAffine(image, rotation_matrix, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return rotated_image

def translate_image(image, tx, ty):
    """
    Translate the image by the specified translation parameters.
    """
    translation_matrix = np.float32([[1, 0, tx], [0, 1, ty]])
    translated_image = cv2.warpAffine(image, translation_matrix, image.shape[1::-1])
    return translated_image

# Example data
#X_train = np.random.rand(100, 28, 28)  # Assuming 100 images of size 28x28
#y_train = np.random.randint(0, 10, 100)  # Assuming 100 labels

# Augmentation parameters
rotation_angles = [20]
translations = [(5, 5)]

augmented_X_train = []
augmented_y_train = []

for image, label in zip(X_train_ct, y_train_ct):
    # Original image
    augmented_X_train.append(image)
    augmented_y_train.append(label)

    # Augment with rotations
    for angle in rotation_angles:
        rotated_image = rotate_image(image, angle)
        augmented_X_train.append(rotated_image)
        augmented_y_train.append(label)

    # Augment with translations
    for tx, ty in translations:
        translated_image = translate_image(image, tx, ty)
        augmented_X_train.append(translated_image)
        augmented_y_train.append(label)

# Convert lists to numpy arrays
augmented_X_train = np.array(augmented_X_train)
augmented_y_train = np.array(augmented_y_train)

# Shuffle the data
shuffle_indices = np.random.permutation(len(augmented_X_train))
augmented_X_train = augmented_X_train[shuffle_indices]
augmented_y_train = augmented_y_train[shuffle_indices]
augmented_X_train.shape, augmented_y_train.shape
# Now, augmented_X_train and augmented_y_train contain the augmented dataset.


((6000, 128, 128, 3), (6000,))

In [20]:

random_indices = np.random.choice(6000, 3712, replace=False)
augmented_X_train = augmented_X_train[random_indices]
augmented_y_train = augmented_y_train[random_indices]
augmented_X_train.shape, augmented_y_train.shape

((3712, 128, 128, 3), (3712,))

In [21]:
labels_train_ct = np.concatenate((y_train_ct, augmented_y_train), axis=0)
images_train_ct = np.concatenate((X_train_ct, augmented_X_train), axis=0)
images_train_ct.shape, labels_train_ct.shape


((5712, 128, 128, 3), (5712,))

In [32]:
'''
X_val.shape, X_test.shape, y_val.shape, y_test.shape
'''

from sklearn.preprocessing import OneHotEncoder
labels_train_ct = labels_train_ct.reshape(-1, 1)
encoder = OneHotEncoder(sparse=False)

# Fit and transform the data
labels_train_ct = encoder.fit_transform(labels_train_ct)
labels_train_ct.shape

(5712, 2)

In [33]:
labels_train_ct.shape, images_train_ct.shape

((5712, 2), (5712, 128, 128, 3))

In [23]:
del augmented_X_train


In [9]:
'''import numpy as np
import cv2

def rotate_image(image, angle):
    """Rotate the image by a specified angle."""
    # Rotate the image using OpenCV's warpAffine function
    height, width = image.shape[:2]
    rotation_matrix = cv2.getRotationMatrix2D((width / 2, height / 2), angle, 1)
    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height))
    return rotated_image

def shift_image(image, shift):
    """Shift the image horizontally and vertically."""
    # Shift the image using OpenCV's warpAffine function
    height, width = image.shape[:2]
    shift_matrix = np.float32([[1, 0, shift[0]], [0, 1, shift[1]]])
    shifted_image = cv2.warpAffine(image, shift_matrix, (width, height))
    return shifted_image

def flip_image(image, flip_direction):
    """Flip the image horizontally or vertically."""
    # Flip the image using OpenCV's flip function
    if flip_direction == 'horizontal':
        flipped_image = cv2.flip(image, 1)
    elif flip_direction == 'vertical':
        flipped_image = cv2.flip(image, 0)
    else:
        flipped_image = image
    return flipped_image

def zoom_image(image, zoom_factor):
    """Zoom into or out of the image."""
    # Zoom the image using OpenCV's resize function
    height, width = image.shape[:2]
    zoomed_height, zoomed_width = int(height * zoom_factor), int(width * zoom_factor)
    zoomed_image = cv2.resize(image, (zoomed_width, zoomed_height))
    return zoomed_image

# Example usage:
# Assuming X_train is your original dataset

# Perform augmentation on each image in X_train
augmented_X_train = []
for image in X_train:
    # Rotate the image by a random angle between -10 and 10 degrees
    rotated_image = rotate_image(image, np.random.uniform(-10, 10))
    
    # Shift the image horizontally and vertically by a random factor between -10 and 10 pixels
    shift = (np.random.uniform(-10, 10), np.random.uniform(-10, 10))
    shifted_image = shift_image(image, shift)
    
    # Add the augmented images to the augmented dataset
    augmented_X_train.extend([rotated_image, shifted_image])

# Convert the list of augmented images to a NumPy array
augmented_X_train = np.array(augmented_X_train)
augmented_X_train.shape'''

'import numpy as np\nimport cv2\n\ndef rotate_image(image, angle):\n    """Rotate the image by a specified angle."""\n    # Rotate the image using OpenCV\'s warpAffine function\n    height, width = image.shape[:2]\n    rotation_matrix = cv2.getRotationMatrix2D((width / 2, height / 2), angle, 1)\n    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height))\n    return rotated_image\n\ndef shift_image(image, shift):\n    """Shift the image horizontally and vertically."""\n    # Shift the image using OpenCV\'s warpAffine function\n    height, width = image.shape[:2]\n    shift_matrix = np.float32([[1, 0, shift[0]], [0, 1, shift[1]]])\n    shifted_image = cv2.warpAffine(image, shift_matrix, (width, height))\n    return shifted_image\n\ndef flip_image(image, flip_direction):\n    """Flip the image horizontally or vertically."""\n    # Flip the image using OpenCV\'s flip function\n    if flip_direction == \'horizontal\':\n        flipped_image = cv2.flip(image, 1)\n    elif fl

In [ ]:
augmented_X_train_ct.shape, augmented_y_train_ct.shape

In [19]:
#images_train, images_train_ct, labels_train_one_hot, labels_train_ct_one_hot, 

In [24]:
'''images_train_ct.shape, labels_train_ct.shape
X_val_ct.shape, X_test_ct.shape, y_val_ct.shape, y_test_ct.shape
X_val_mri.shape, X_test_mri.shape, y_val_mri.shape, y_test_mri.shape
images_train.shape, labels_train_mri.shape
X_train_ct.shape, X_test_ct.shape, y_train_ct.shape, y_test_ct.shape'''

(1311, 4)

### proposede Method to perform Teacher model

### GSCA layer

In [24]:
import tensorflow as tf
from tensorflow.keras import layers

class DeeperGlobalLocalAttentionLayer(layers.Layer):
    def __init__(self, units, activation='sigmoid', dropout_rate=0.2, use_scale=True, **kwargs):
        super(DeeperGlobalLocalAttentionLayer, self).__init__(**kwargs)
        self.units = units
        self.activation = activation
        self.dropout_rate = dropout_rate
        self.use_scale = use_scale

    def build(self, input_shape):
        _, _, _, channels = input_shape
        self.global_avg_pooling = layers.GlobalAveragePooling2D()
        self.global_attention = layers.Dense(units=self.units, activation=self.activation)
        
        self.local_conv1 = layers.Conv2D(filters=self.units, kernel_size=(1, 1), activation=self.activation)
        self.local_conv2 = layers.Conv2D(filters=self.units, kernel_size=(1, 1), activation=self.activation)
        
        if self.use_scale:
            self.global_scale = self.add_weight(shape=(1, 1, 1, 1), initializer='ones', trainable=True, name='global_scale')
            self.local_scale = self.add_weight(shape=(1, 1, 1, self.units), initializer='ones', trainable=True, name='local_scale')
        
        super(DeeperGlobalLocalAttentionLayer, self).build(input_shape)

    def call(self, inputs, training=None):
        # Deeper Global Attention
        global_avg = self.global_avg_pooling(inputs)
        global_attention = self.global_attention(global_avg)
        global_attention = tf.expand_dims(tf.expand_dims(global_attention, 1), 1)

        # Deeper Spatial Attention
        local_attention1 = self.local_conv1(inputs)
        local_attention1 = tf.reduce_mean(local_attention1, axis=[1, 2], keepdims=True)  # Reduce spatial dimensions
        local_attention2 = self.local_conv2(local_attention1)
        local_attention = tf.reduce_mean(local_attention2, axis=[1, 2], keepdims=True)  # Reduce spatial dimensions

        # Scale Global and Spatial Attention
        if self.use_scale:
            global_attention *= self.global_scale
            local_attention *= self.local_scale

        # Combine Global and Spatial Attention
        attention = tf.sigmoid(global_attention + local_attention)
        return attention

    def get_config(self):
        config = super(DeeperGlobalLocalAttentionLayer, self).get_config()
        config.update({'units': self.units, 'activation': self.activation, 'dropout_rate': self.dropout_rate,
                       'use_scale': self.use_scale})
        return config

class DeeperAttentionLayer(layers.Layer):
    def __init__(self, units=64, use_scale=True, **kwargs):
        super(DeeperAttentionLayer, self).__init__(**kwargs)
        self.units = units
        self.use_scale = use_scale

    def build(self, input_shape):
        _, H, W, C = input_shape
        ## add channel wise weights to performa as channel attention
        self.alpha = self.add_weight(shape=(1, 1, 1, C), initializer='ones', trainable=True, name='alpha')
        self.deeper_global_local_attention = DeeperGlobalLocalAttentionLayer(units=self.units, activation='sigmoid', 
                                                                              dropout_rate=0.2,  # You can adjust the dropout rate
                                                                              use_scale=self.use_scale)
        super(DeeperAttentionLayer, self).build(input_shape)

    def call(self, inputs, training=None):
        
        attention = self.deeper_global_local_attention(inputs, training=training)
        attention_feature = inputs * attention * self.alpha
        return attention_feature

    def get_config(self):
        config = super(DeeperAttentionLayer, self).get_config()
        config.update({'units': self.units, 'use_scale': self.use_scale})
        return config


In [27]:
from tensorflow.keras.models import load_model

# Define custom objects dictionary
custom_objects = {
    'DeeperAttentionLayer': DeeperAttentionLayer,
    ##'distillation_loss': distillation_loss
}
import shutil

# Source path of the model file (read-only directory)
source_path = '/kaggle/input/best-teacher-model-brain/best_teacher1_model_brain.keras'

# Destination path in the writable directory
destination_path = '/kaggle/working/best_teacher1_model_brain.keras'

# Copy the model file from the source path to the destination path
shutil.copyfile(source_path, destination_path)

# Load the model with custom objects
model1 = load_model('/kaggle/working/best_teacher1_model_brain.keras', 
                    custom_objects={'DeeperAttentionLayer':DeeperAttentionLayer})

# Now you can use the loaded model for evaluation or further training
model1.evaluate([images_test, images_test_ct], [labels_test_one_hot, labels_test_ct_one_hot])

 3/41 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - dense_98_accuracy: 0.9931 - dense_99_accuracy: 1.0000 - loss: 0.0380  

I0000 00:00:1709544930.020553     439 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


41/41 ━━━━━━━━━━━━━━━━━━━━ 28s 257ms/step - dense_98_accuracy: 0.9803 - dense_99_accuracy: 0.9997 - loss: 0.1163


[0.19087521731853485, 0.9748283624649048, 0.9992372393608093]

In [29]:
from tensorflow.keras.models import load_model

# Define custom objects dictionary
custom_objects = {
    'DeeperAttentionLayer': DeeperAttentionLayer,
    ##'distillation_loss': distillation_loss
}
import shutil

# Source path of the model file (read-only directory)
source_path = '/kaggle/input/best-teacher-model-brain/best_teacher1_model_brain.keras'

# Destination path in the writable directory
destination_path = '/kaggle/working/best_teacher1_model_brain.keras'

# Copy the model file from the source path to the destination path
shutil.copyfile(source_path, destination_path)

# Load the model with custom objects
model1 = load_model('/kaggle/working/best_teacher1_model_brain.keras', 
                    custom_objects={'DeeperAttentionLayer':DeeperAttentionLayer})

# Now you can use the loaded model for evaluation or further training
model1.evaluate([images_test, images_test_ct], [labels_test_one_hot, labels_test_ct_one_hot])


 2/41 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - dense_98_accuracy: 1.0000 - dense_99_accuracy: 1.0000 - loss: 0.0100  

I0000 00:00:1709278834.755392     116 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


41/41 ━━━━━━━━━━━━━━━━━━━━ 28s 255ms/step - dense_98_accuracy: 0.9803 - dense_99_accuracy: 1.0000 - loss: 0.1113


[0.18918003141880035, 0.9748283624649048, 1.0]

In [29]:
import tensorflow as tf

# Print TensorFlow version
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.15.0


In [46]:
def RGSCA(x, filters, strides=(1, 1), use_projection=False):
    shortcut = x

    # Define the first convolutional layer of the block
    x = Conv2D(filters=filters, kernel_size=(3, 3), strides=strides, padding='same', 
               #activation = 'relu'
              )(x)
    x = DeeperAttentionLayer(units=filters, use_scale=True)(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    # Define the second convolutional layer of the block
    x = Conv2D(filters=filters, kernel_size=(3, 3), padding='same')(x)
    x = DeeperAttentionLayer(units=filters, use_scale=True)(x)
    
    x = BatchNormalization()(x)

    # If the stride is not (1, 1), the dimensions need to be adjusted
    if strides != (1, 1) or use_projection:
        
        shortcut = Conv2D(filters=filters, kernel_size=(1, 1), strides=strides, padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)

    # Add the shortcut (identity connection)
    
    x = tf.keras.layers.add([x, shortcut])
    
    x = tf.keras.layers.Activation('relu')(x)
    return x

#def build_resnet18(input_shape=(128, 128, 3), num_classes=2):
input_shape=(128, 128, 3)
inputs = Input(shape=input_shape)
inputs2 = Input(shape=input_shape)
#input_data = Input(shape=input_shape, name='input_data')
# Initial convolutional layer


x = Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2), padding='same')(inputs)
x = DeeperAttentionLayer(units=64, use_scale=True)(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

x = RGSCA(x, filters=64)
x = DeeperAttentionLayer(units=64, use_scale=True)(x) ## GSCA layer

x = RGSCA(x, filters=64)
x = tf.keras.layers.Dropout(0.2)(x)

x = RGSCA(x, filters=64)
x = DeeperAttentionLayer(units=64, use_scale=True)(x) ## GSCA layer

x = RGSCA(x, filters=128, strides=(2, 2), use_projection=True)
x = DeeperAttentionLayer(units=128, use_scale=True)(x)

x = RGSCA(x, filters=128)
x = tf.keras.layers.Dropout(0.2)(x)

x = RGSCA(x, filters=128)
x = DeeperAttentionLayer(units=128, use_scale=True)(x) ## GSCA layer

x = RGSCA(x, filters=256, strides=(2, 2), use_projection=True)
x = DeeperAttentionLayer(units=256, use_scale=True)(x)

x = RGSCA(x, filters=256)
x = tf.keras.layers.Dropout(0.2)(x)

x = RGSCA(x, filters=256)
x = DeeperAttentionLayer(units=256, use_scale=True)(x) ## GSCA layer

x = RGSCA(x, filters=512, strides=(2, 2), use_projection=True)
x = DeeperAttentionLayer(units=512, use_scale=True)(x)

x = RGSCA(x, filters=512)
x = DeeperAttentionLayer(units=512, use_scale=True)(x)


x2 = Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2), padding='same')(inputs2)
x2 = DeeperAttentionLayer(units=64, use_scale=True)(x2)
x2 = BatchNormalization()(x2)
x2 = tf.keras.layers.Activation('relu')(x2)
x2 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x2)

x2 = RGSCA(x2, filters=64)
x2 = DeeperAttentionLayer(units=64, use_scale=True)(x2) ## GSCA layer

x2 = RGSCA(x2, filters=64)
x2 = tf.keras.layers.Dropout(0.2)(x2)

x2 = RGSCA(x2, filters=64)
x2 = DeeperAttentionLayer(units=64, use_scale=True)(x2) ## GSCA layer

x2 = RGSCA(x2, filters=128, strides=(2, 2), use_projection=True)
x2 = DeeperAttentionLayer(units=128, use_scale=True)(x2)

x2 = RGSCA(x2, filters=128)
x2 = tf.keras.layers.Dropout(0.2)(x2)

x2 = RGSCA(x2, filters=128)
x2 = DeeperAttentionLayer(units=128, use_scale=True)(x2) ## GSCA layer

x2 = RGSCA(x2, filters=256, strides=(2, 2), use_projection=True)
x2 = DeeperAttentionLayer(units=256, use_scale=True)(x2)

x2 = RGSCA(x2, filters=256)
x2 = tf.keras.layers.Dropout(0.2)(x2)

x2 = RGSCA(x2, filters=256)
x2 = DeeperAttentionLayer(units=256, use_scale=True)(x2) ## GSCA layer

x2 = RGSCA(x2, filters=512, strides=(2, 2), use_projection=True)
x2 = DeeperAttentionLayer(units=512, use_scale=True)(x2)

x2 = RGSCA(x2, filters=512)
x2 = DeeperAttentionLayer(units=512, use_scale=True)(x2)

# Global average pooling and fully connected layer
con = tf.keras.layers.Concatenate(axis=-1)([x, x2])

x = GlobalAveragePooling2D()(con)
x2 = tf.keras.layers.GlobalMaxPooling2D()(con)

conx = tf.keras.layers.Add()([x, x2])
conx = tf.keras.layers.ReLU()(conx)

##x = Dense(128, activation='selu')(x)
outputs1 = Dense(4, activation='softmax')(conx)
outputs2 = Dense(2, activation='sigmoid')(conx)

# Create the model
teacher_model1 = Model([inputs, inputs2], [outputs1, outputs2])
#return model

# Instantiate the ResNet-18 model
#model = build_resnet18()


#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Train the model
#model.fit(images_train, y_train_one_hot, epochs=10, batch_size=32, validation_split=0.2)
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
def checkpoint_callback():

    checkpoint_filepath = 'best_models_teacer_1.keras'

    model_checkpoint_callback= ModelCheckpoint(filepath=checkpoint_filepath,
                           save_weights_only=False,
                           #frequency='epoch',
                           monitor='val_loss',
                           save_best_only=True,
                           verbose=1)

    return model_checkpoint_callback

def early_stopping(patience):
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, verbose=1)
    return es_callback



checkpoint_callback = checkpoint_callback()
early_stopping = early_stopping(patience=100)
callbacks = [checkpoint_callback, early_stopping]
#model.summary()
teacher_model1.compile(optimizer='adam', loss=['categorical_crossentropy','binary_crossentropy'], 
                                      metrics=['accuracy', 'accuracy'])

'''
images_train_ct.shape, labels_train_ct.shape
X_val_ct.shape, X_test_ct.shape, y_val_ct.shape, y_test_ct.shape
X_val_mri.shape, X_test_mri.shape, y_val_mri.shape, y_test_mri.shape
images_train.shape, labels_train_mri.shape
X_train_ct.shape, X_test_ct.shape, y_train_ct.shape, y_test_ct.shape
'''
print('images_train:', images_train.shape)
print('images_train_ct:', images_train_ct.shape)

print('labels_train_mri:', labels_train_mri.shape)
print('labels_train_ct:', labels_train_ct.shape)

print('X_val_mri:', X_val_mri.shape)
print('X_val_ct:', X_val_ct.shape)

print('y_val_mri:', y_val_mri.shape)
print('y_val_ct:', y_val_ct.shape)


teacher_model1.fit(
    [images_train, images_train_ct], [labels_train_mri, labels_train_ct],
    epochs=200,
    validation_data=([X_val_mri, X_val_ct], [y_val_mri, y_val_ct]),
    callbacks=callbacks,
    #save_freq='epoch'  # This specifies saving the model after each epoch
)


images_train: (5712, 128, 128, 3)
images_train_ct: (5712, 128, 128, 3)
labels_train_mri: (5712, 4)
labels_train_ct: (5712, 2)
X_val_mri: (655, 128, 128, 3)
X_val_ct: (655, 128, 128, 3)
y_val_mri: (655, 4)
y_val_ct: (655, 2)
Epoch 1/200
178/179 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - dense_152_accuracy: 0.5284 - dense_153_accuracy: 0.6060 - loss: 2.6803

2024-03-11 12:34:20.742673: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 6m28.225406455s

********************************
[Compiling module a_inference_one_step_on_data_1095339__.85706] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************


179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - dense_152_accuracy: 0.5289 - dense_153_accuracy: 0.6062 - loss: 2.6758   
Epoch 1: val_loss improved from inf to 9.67936, saving model to best_models_teacer_1.keras
179/179 ━━━━━━━━━━━━━━━━━━━━ 803s 3s/step - dense_152_accuracy: 0.5295 - dense_153_accuracy: 0.6063 - loss: 2.6714 - val_dense_152_accuracy: 0.2275 - val_dense_153_accuracy: 0.6076 - val_loss: 9.6794
Epoch 2/200
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - dense_152_accuracy: 0.7662 - dense_153_accuracy: 0.7091 - loss: 1.2084
Epoch 2: val_loss improved from 9.67936 to 2.35239, saving model to best_models_teacer_1.keras
179/179 ━━━━━━━━━━━━━━━━━━━━ 29s 160ms/step - dense_152_accuracy: 0.7664 - dense_153_accuracy: 0.7092 - loss: 1.2080 - val_dense_152_accuracy: 0.5756 - val_dense_153_accuracy: 0.4107 - val_loss: 2.3524
Epoch 3/200
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - dense_152_accuracy: 0.8546 - dense_153_accuracy: 0.7786 - loss: 0.8524
Epoch 3: val_loss improved from 2.35239 

In [47]:
teacher_model1.evaluate([X_test_mri, X_test_ct], [y_test_mri, y_test_ct])

21/21 ━━━━━━━━━━━━━━━━━━━━ 8s 420ms/step - dense_152_accuracy: 0.9953 - dense_153_accuracy: 0.9973 - loss: 0.0258


[0.05517842620611191, 0.9908536672592163, 0.9908536672592163]

In [45]:
teacher_model1.evaluate([X_test_mri, X_test_ct], [y_test_mri, y_test_ct])

21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - dense_44_accuracy: 0.9919 - dense_45_accuracy: 0.9986 - loss: 0.0406


[0.04907708615064621, 0.9923780560493469, 0.9969512224197388]

In [41]:
'''
[images_train, images_train_ct], [labels_train_mri, labels_train_ct],
    epochs=200,
    validation_data=([X_val_mri, X_val_ct], [y_val_mri, y_val_ct]),
    
'''
np.save('X_test_mri.npy', X_test_mri)
np.save('X_test_ct.npy', X_test_ct)
np.save('y_test_mri.npy', y_test_mri)
np.save('y_test_ct.npy', y_test_ct)

np.save('images_train.npy', images_train)
np.save('images_train_ct.npy', images_train_ct)
np.save('labels_train_mri.npy', labels_train_mri)
np.save('labels_train_ct.npy', labels_train_ct)

np.save('X_val_mri.npy', X_val_mri)
np.save('X_val_ct.npy', X_val_ct)
np.save('y_val_mri.npy', y_val_mri)
np.save('y_val_ct.npy', y_val_ct)

In [49]:
teacher_model1.save('second_best_ever_teacher1_model.keras')

In [48]:
from tensorflow.keras.models import load_model

# Load the model
teacher_model1_1 = load_model('/kaggle/working/best_models_teacer_1.keras', 
                              custom_objects={'DeeperAttentionLayer':DeeperAttentionLayer})
teacher_model1_1.evaluate([X_test_mri, X_test_ct], [y_test_mri, y_test_ct],
               #batch_size=128
                         )

21/21 ━━━━━━━━━━━━━━━━━━━━ 25s 408ms/step - dense_152_accuracy: 0.9934 - dense_153_accuracy: 0.9914 - loss: 0.0304


[0.03463848680257797, 0.9939024448394775, 0.9908536672592163]

In [50]:
teacher_model1_1.save('first_best_ever_teacher1_model.keras')

In [51]:
y_pred = teacher_model1_1.predict([X_test_mri, X_test_ct])
#[X_test_mri, X_test_ct], [y_test_mri, y_test_ct]
y_pred_binary = y_pred[0] >= 0.5
y_pred_binary_pgd_test = np.array(y_pred_binary, dtype='int32')

print('y_pred_binary_pgd_test:', y_pred_binary_pgd_test.shape)
# Calculate evaluation metrics for the current epsilon
y_test_categorical = y_test_mri
y_test_categorical_c = y_test_ct

accuracy = accuracy_score(y_pred_binary_pgd_test, y_test_categorical) * 100
precision = precision_score(y_pred_binary_pgd_test, y_test_categorical, average='macro') * 100
recall = recall_score(y_pred_binary_pgd_test, y_test_categorical, average='macro') * 100
f1 = f1_score(y_pred_binary_pgd_test, y_test_categorical, average='macro') * 100
#auc = roc_auc_score(y_pred, y_train_categorical, multi_class='ovr') * 100
print('accuracy:', accuracy)
print('precision:', precision)
print('recall:', recall)
print('f1:', f1)

y_pred_binary = y_pred[1] >= 0.5
y_pred_binary_pgd_test_c = np.array(y_pred_binary, dtype='int32')

print('y_pred_binary_pgd_test_c:', y_pred_binary_pgd_test_c.shape)
# Calculate evaluation metrics for the current epsilon
accuracy = accuracy_score(y_pred_binary_pgd_test_c, y_test_categorical_c) * 100
precision = precision_score(y_pred_binary_pgd_test_c, y_test_categorical_c, average='macro') * 100
recall = recall_score(y_pred_binary_pgd_test_c, y_test_categorical_c, average='macro') * 100
f1 = f1_score(y_pred_binary_pgd_test_c, y_test_categorical_c, average='macro') * 100
print('accuracy:', accuracy)
print('precision:', precision)
print('recall:', recall)
print('f1:', f1)


21/21 ━━━━━━━━━━━━━━━━━━━━ 27s 692ms/step
y_pred_binary_pgd_test: (656, 4)
accuracy: 99.39024390243902
precision: 99.36399612340495
recall: 99.34883327238992
f1: 99.35480609995224
y_pred_binary_pgd_test_c: (656, 2)
accuracy: 99.08536585365853
precision: 98.94845034788109
recall: 99.10569105691056
f1: 99.02596438434811


## Generating soft predictions of tasks 1 and 2 for student model by the teacher1

In [54]:
y_pred = teacher_model1_1.predict([images_train, images_train_ct])
#[images_train, images_train_ct], [labels_train_mri, labels_train_ct]
y_pred_binary = y_pred[0] >= 0.5
soft_pred_train_mri = np.array(y_pred_binary, dtype='int32')

## generating soft probabilities for task 1 of teacher 1: MRI images for binary class classification
## for student model

print('soft_pred_train_mri:', soft_pred_train_mri.shape)

## original lab=bels for MRI images
y_train_categorical = labels_train_mri
## original lab=bels for Ct scan images
y_train_categorical_c = labels_train_ct

accuracy = accuracy_score(soft_pred_train_mri, y_train_categorical) * 100
precision = precision_score(soft_pred_train_mri, y_train_categorical, average='macro') * 100
recall = recall_score(soft_pred_train_mri, y_train_categorical, average='macro') * 100
f1 = f1_score(soft_pred_train_mri, y_train_categorical, average='macro') * 100
#auc = roc_auc_score(y_pred, y_train_categorical, multi_class='ovr') * 100
print('accuracy:', accuracy)
print('precision:', precision)
print('recall:', recall)
print('f1:', f1)

y_pred_binary = y_pred[1] >= 0.5
y_pred_ctscan = np.array(y_pred_binary, dtype='int32')

## generating soft probabilities for task 2 of teacher 1: CT scan images for binary class classification
## for student model
print('y_pred_ctscan:', y_pred_ctscan.shape)
# Calculate evaluation metrics for the current epsilon
accuracy = accuracy_score(y_pred_ctscan, y_train_categorical_c) * 100
precision = precision_score(y_pred_ctscan, y_train_categorical_c, average='macro') * 100
recall = recall_score(y_pred_ctscan, y_train_categorical_c, average='macro') * 100
f1 = f1_score(y_pred_ctscan, y_train_categorical_c, average='macro') * 100
print('accuracy:', accuracy)
print('precision:', precision)
print('recall:', recall)
print('f1:', f1)


179/179 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step
soft_pred_train_mri: (5712, 4)
accuracy: 100.0
precision: 100.0
recall: 100.0
f1: 100.0
y_pred_ctscan: (5712, 2)
accuracy: 99.9124649859944
precision: 99.91613841706952
recall: 99.9206313125573
f1: 99.91836518485613


In [55]:
soft_pred_train_mri.shape, y_pred_ctscan.shape

((5712, 4), (5712, 2))

In [57]:
np.save('soft_pred_train_teacher1_brain_mri.npy', soft_pred_train_mri)
np.save('soft_pred_train_teacher1_brain_ctscan.npy', y_pred_ctscan)

In [52]:
y_pred = teacher_model1.predict([X_test_mri, X_test_ct])
#[X_test_mri, X_test_ct], [y_test_mri, y_test_ct]
y_pred_binary = y_pred[0] >= 0.5
y_pred_binary_pgd_test = np.array(y_pred_binary, dtype='int32')

print('y_pred_binary_pgd_test:', y_pred_binary_pgd_test.shape)
# Calculate evaluation metrics for the current epsilon
y_test_categorical = y_test_mri
y_test_categorical_c = y_test_ct

accuracy = accuracy_score(y_pred_binary_pgd_test, y_test_categorical) * 100
precision = precision_score(y_pred_binary_pgd_test, y_test_categorical, average='macro') * 100
recall = recall_score(y_pred_binary_pgd_test, y_test_categorical, average='macro') * 100
f1 = f1_score(y_pred_binary_pgd_test, y_test_categorical, average='macro') * 100
#auc = roc_auc_score(y_pred, y_train_categorical, multi_class='ovr') * 100
print('accuracy:', accuracy)
print('precision:', precision)
print('recall:', recall)
print('f1:', f1)

y_pred_binary = y_pred[1] >= 0.5
y_pred_binary_pgd_test_c = np.array(y_pred_binary, dtype='int32')

print('y_pred_binary_pgd_test_c:', y_pred_binary_pgd_test_c.shape)
# Calculate evaluation metrics for the current epsilon
accuracy = accuracy_score(y_pred_binary_pgd_test_c, y_test_categorical_c) * 100
precision = precision_score(y_pred_binary_pgd_test_c, y_test_categorical_c, average='macro') * 100
recall = recall_score(y_pred_binary_pgd_test_c, y_test_categorical_c, average='macro') * 100
f1 = f1_score(y_pred_binary_pgd_test_c, y_test_categorical_c, average='macro') * 100
print('accuracy:', accuracy)
print('precision:', precision)
print('recall:', recall)
print('f1:', f1)


21/21 ━━━━━━━━━━━━━━━━━━━━ 27s 674ms/step
y_pred_binary_pgd_test: (656, 4)
accuracy: 99.08536585365853
precision: 99.0722419641415
recall: 99.02688172043011
f1: 99.03919497741008
y_pred_binary_pgd_test_c: (656, 2)
accuracy: 99.08536585365853
precision: 99.18564199873498
recall: 98.88417413169887
f1: 99.03054187192117
